In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob as tb
import langid
import os
import re
import nltk
from nltk.stem import RSLPStemmer

In [2]:
def orengo(song):
    st = RSLPStemmer()
    words = filter(lambda x: len(x)>0, song)
    word_list = list(map(lambda x: st.stem(x), words))
    return word_list

In [3]:
def remove_stopwords(doc):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    new_doc = list(filter(lambda x: x not in stopwords, doc))
    return new_doc

In [4]:
df = pd.read_csv('.\data\dataset_lyrics.csv')

In [5]:
#removes compilations
df = df[(df.artist != '/colecao-amo-voce/')&(df.artist != '/corinhos-infantis/')&(df.artist != '/corinhos-evangelicos/')&
        (df.artist != '/harpa-crista/')&(df.artist != '/hinos/')&(df.artist != '/musicas-catolicas/')&(df.artist != '/musicas-gospel/')&
        (df.artist != '/pineapple/')&(df.artist != '/umbanda/')&(df.artist != '/alok/')]

In [6]:
#removes artists with less than 25 songs
df = df[(df.artist != '/melim/')&(df.artist != '/kell-smith/')&(df.artist != '/midiam-lima/')&(df.artist != '/isadora-pompeo/')]

In [7]:
df.describe()

,artist,music_name,lyrics
count,2163,2163,2163
unique,87,2163,2163
top,/thiaguinho/,/roupa-nova/whisky-a-go-go.html,"<div data-plugin=""googleTranslate"" id=""lyrics""..."
freq,25,1,1


In [8]:
#data cleansing
df['lyrics'] = df['lyrics'].str.strip('<div data-plugin=""googleTranslate"" id=""lyrics"">')
df['lyrics'] = df['lyrics'].str.strip('<img alt="Instrumental" class="instrumental-icon" src="/img/etc/instrumental.png"/')
df['lyrics'] = df['lyrics'].apply(lambda x: x.lower())

In [9]:
df['lyrics'] = df['lyrics'].str.replace('\n', ' ')
#df['lyrics'] = df['lyrics'].str.replace('\r', '')
df['lyrics'] = df['lyrics'].apply(lambda x: re.sub('[0-9!,.;:\]?}{()"["|@#$%*]', "", x))

In [10]:
#creates new df where the column lyrics carries every song of a given artist
df1 = df.groupby('artist')['lyrics'].apply(' '.join).reset_index()

In [11]:
#applies stemmer and creates new column
df1['orengo'] = df1['lyrics'].apply(lambda x: orengo(x.split(' ')))

In [12]:
#removes stopwords and creates new column
df1['no_stopwords'] = df1['lyrics'].apply(lambda x: remove_stopwords(x.split(' ')))

In [ ]:
#removes stopwords from translated text and creates new column
df1['ns_pt'] = df1['translated'].apply(lambda x: remove_stopwords(x.split(' ')))

In [13]:
#vocabulary
df1['unique'] = df1['lyrics'].apply(lambda x: len(set(x.split(' '))))

In [14]:
#vocabulary after stemming
df1['orengo_unique'] = df1['orengo'].apply(lambda x: len(set(x)))

In [15]:
#vocabulary without considering stopwords
df1['ns_unique'] = df1['no_stopwords'].apply(lambda x: len(set(x)))

In [16]:
#vocabulary without considering stopwords after stemming
df1['ns_orengo_unique'] = df1['no_stopwords'].apply(lambda x: len(set(orengo(x))))

In [ ]:
#vocabulary after translation
df1['unique_pt'] = df['translated'].apply(lambda x: len(set(x.split(' '))))

In [ ]:
#vocabulary after stemming + translation
df1['orengo_pt'] = df['translated'].apply(lambda x: len(set(x)))

In [ ]:
#vocabulary without considering stopwords (translated)
df1['ns_uni_pt'] = df1['ns_pt'].apply(lambda x: len(set(x)))

In [ ]:
#vocabulary without considering stopwords after stemming + translation
df1['ns_or_pt'] = df1['ns_pt'].apply(lambda x: len(set(orengo(x))))

In [17]:
df1.head()

,artist,lyrics,orengo,no_stopwords,unique,orengo_unique,ns_unique,ns_orengo_unique
0,/aline-barros/,mestre eu preciso de um milagre transforma min...,"[mestr, eu, precis, de, um, milagr, transform,...","[mestre, preciso, milagre, transforma, vida, e...",748,581,651,520
1,/amado-batista/,ao te ver pela primeira vez eu tremi todo uma ...,"[ao, te, ver, pel, prim, vez, eu, trem, tod, u...","[ver, primeira, vez, tremi, todo, coisa, tomou...",846,621,750,559
2,/ana-carolina/,eu e você não é assim tão complicado não é dif...,"[eu, e, voc, não, é, assim, tão, complic, não,...","[assim, tão, complicado, difícil, perceber, , ...",776,599,686,539
3,/anavitoria/,eu poderia acordar sem teu olhar de sono sem t...,"[eu, pod, acord, sem, teu, olh, de, son, sem, ...","[poderia, acordar, olhar, sono, lábio, dono, q...",852,673,767,613
4,/anderson-freire/,não consigo ir além do teu olhar tudo que eu c...,"[não, consig, ir, além, do, teu, olh, tud, que...","[consigo, ir, além, olhar, tudo, consigo, imag...",1127,868,1025,801


In [18]:
df1.to_csv('./data/final_data.csv', index=False)